In [68]:
#1. Copy raw data to data/temp folder 
dir.create("data/temp")

data.path <- paste0("data/raw/", list.files("data/raw/"))


In [69]:
#2. Unzip raw files in data/temp folder 
for (file in data.path){
    file.dir <- gsub(".zip", "", file)
    file.dir <- gsub("raw", "temp", file.dir)
    unzip(zipfile = file, exdir = file.dir) 
}

In [70]:
data.path.temp <- paste0("data/temp/", list.files("data/temp"), "/raw_sequences")

md5dfs <- c()
for (folder in data.path.temp) {
    namesFiles <- list.files(path = folder)
    # Select only names of fastq.gz files 
    namesFilesGz <- namesFiles[grepl(".gz", namesFiles)]

    # Calculate md5 sums
    md5 <- c()
    SampleID <- c()
    for (i in namesFilesGz) {
        f.name <- paste(folder, i, sep="/")
        md <- tools::md5sum(f.name)
        md5 <- c(md5, md)
        SampleID <- c(SampleID, i) 
        }
    
newfilesMD5 <- data.frame(SampleID, md5)
md5dfs <- rbind(md5dfs, newfilesMD5) }


In [71]:
# Read in and prepare md5 information provided by BaseClear
###########################################################

md5.files <- list.files(path = "data/temp", pattern = "md5", recursive = TRUE)

md5.run1 <- rbind(read.delim(paste0("data/temp/", md5.files[1]), sep = " ", header = FALSE), 
             read.delim(paste0("data/temp/", md5.files[2]), sep = " ", header = FALSE))

md5.run2 <- read.delim(paste0("data/temp/", md5.files[5]), sep = " ", header = FALSE)

md5.run3 <- read.delim(paste0("data/temp/", md5.files[3]), sep = "\t", header = FALSE)
md5.run3$V2 <- sub(".*/", "", md5.run3$V2)

md5.run123 <- cbind(c(as.character(md5.run1$V1), as.character(md5.run2$V1), 
                      as.character(md5.run3$V1)), 
                    c(as.character(md5.run1$V3), as.character(md5.run2$V3), 
                      as.character(md5.run3$V2)))



In [72]:
# Compare created fasta files with provided by BaseClear file 
#############################################################

md5dfs <- md5dfs[order(as.character(md5dfs$SampleID)), ]
md5.run123 <- md5.run123[order(md5.run123[,2]), ]

table(as.character(md5dfs$md5) == md5.run123[, 1])


TRUE 
 558 

In [73]:
# Create directory Dada2_input and move there all files 
unlink("data/Dada2_input", recursive=TRUE)

dir.create("data/Dada2_input")

gz.all <- list.files("data/temp/", pattern = ".fastq.gz", recursive = TRUE)

gz.all <- paste0("data/temp/", gz.all)

file.copy(from = gz.all, to = "data/Dada2_input")

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[136] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[151] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[166] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[181] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[196] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[211] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[226] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[241] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[256] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[271] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[286] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[301] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[316] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[331] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[346] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[361] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[376] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[391] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[406] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[421] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[436] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[451] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[466] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[481] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[496] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[511] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[526] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[541] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[556] TRUE TRUE TRUE

In [74]:
# Remove temp folder 
unlink("data/temp", recursive=TRUE)


In [75]:
# Read in metadata and list files 
metadata.f <- read.csv("data/metadata/samples_metadata.csv")

fastFiles <- list.files("data/Dada2_input/")

length(fastFiles)

[1] 558

In [76]:
# Remove sample that didn't pass control 
########################################

s.remove <- metadata.f[metadata.f$FilesKeep %in% "Remove", ]

for (f in s.remove$OriginalID) {
    unlink(paste0("data/Dada2_input/", fastFiles[grep(f, fastFiles)]))
}


In [77]:
# Replace all "-" with "_"
##########################
fastFiles <- list.files("data/Dada2_input/")

for (i in 1:length(fastFiles)) {
    
    f1 <- paste0("data/Dada2_input/", fastFiles[i])
    
    f2 <- paste0("data/Dada2_input/", gsub("-", "_", fastFiles[i])) 
   
    file.rename(from = f1, to = f2)
}


In [79]:
metadata.f1$OriginalID <- gsub("-", "_", metadata.f1$OriginalID)
metadata.f1$OriginalID 

[1] "1351_10_12_2001"           "1351_13_5_2002"           
  [3] "1351_14_8_2000"            "1351_16_10_2002"          
  [5] "1351_17_3_2002"            "1351_17_9_2001"           
  [7] "1351_18_5_1999"            "1351_2_9_2002"            
  [9] "1351_27_9_1999"            "1351_4_3_2003"            
 [11] "1351_5_5_2003"             "1351_6_8_2001"            
 [13] "1351_7_5_2001"             "1351_9_1_2001"            
 [15] "1356_14_5_02"              "1356_15_10_02"            
 [17] "1356_15_8_00"              "1356_17_5_99"             
 [19] "1356_18_4_00"              "1356_18_9_01"             
 [21] "1356_19_03_02"             "1356_23_01_01"            
 [23] "1356_3_9_02"               "1356_4_2_03"              
 [25] "1356_6_5_03"               "1356_7_8_01"              
 [27] "1356_8_5_01"               "1359_14_5_01"             
 [29] "1359_15_10_02"             "1359_15_8_00"             
 [31] "1359_17_5_99"              "1359_18_9_01"             
 [33] "1359_19_3_02"              "1359_21_8_01"             
 [35] "1359_23_01_01"             "1359_3_9_02"              
 [37] "1359_4_2_03"               "1359_6_5_03"              
 [39] "1359_8_5_01"               "1359_8_9_99"              
 [41] "1360_14_5_02"              "1360_15_10_02"            
 [43] "1360_15_2_1999"            "1360_17_05_99"            
 [45] "1360_18_09_01"             "1360_19_5_03"             
 [47] "1360_23_1_1"               "1360_3_9_02"              
 [49] "1360_4_2_2003"             "1360_4_9_01"              
 [51] "1360_6_5_03"               "1360_7_8_01"              
 [53] "1360_8_5_01"               "1363_10_12_2001"          
 [55] "1363_13_5_2002"            "1363_14_8_2000"           
 [57] "1363_16_10_2002"           "1363_17_3_2002"           
 [59] "1363_17_9_2001"            "1363_18_5_1999"           
 [61] "1363_2_9_2002"             "1363_27_9_1999"           
 [63] "1363_4_3_2003"             "1363_5_5_2003"            
 [65] "1363_6_8_2001"             "1363_7_5_2001"            
 [67] "1363_8_2_1999"             "1363_9_1_2001"            
 [69] "1364_10_12_2001"           "1364_13_5_2002"           
 [71] "1364_14_8_2000"            "1364_16_10_2002"          
 [73] "1364_17_3_2002"            "1364_17_4_2000"           
 [75] "1364_17_9_2001"            "1364_18_5_1999"           
 [77] "1364_2_9_2002"             "1364_27_9_1999"           
 [79] "1364_4_3_2003"             "1364_5_5_2003"            
 [81] "1364_6_8_2001"             "1364_7_5_2001"            
 [83] "1364_8_2_1999"             "1364_9_1_2001"            
 [85] "1999_02_22_1348"           "1999_02_22_1352"          
 [87] "1999_02_22_1353"           "1999_02_22_1354"          
 [89] "1999_02_22_1355"           "1999_02_22_1356"          
 [91] "1999_02_22_1357"           "1999_02_22_1358"          
 [93] "1999_02_22_1359"           "1999_02_22_1361"          
 [95] "1999_02_22_1362"           "1999_02_22_1363"          
 [97] "1999_02_22_1364"           "1999_02_22_1365"          
 [99] "1999_02_22_1366"           "1999_02_22_1367"          
[101] "1999_05_04_1348"           "1999_05_04_1349"          
[103] "1999_05_04_1350"           "1999_05_04_1351"          
[105] "1999_05_04_1352"           "1999_05_04_1353"          
[107] "1999_05_04_1354"           "1999_05_04_1356"          
[109] "1999_05_04_1357"           "1999_05_04_1358"          
[111] "1999_05_04_1359"           "1999_05_04_1360"          
[113] "1999_05_04_1361"           "1999_05_04_1362"          
[115] "1999_05_04_1363"           "1999_05_04_1364"          
[117] "1999_05_04_1365"           "1999_05_04_1366"          
[119] "1999_05_04_1367"           "1999_06_15_1348"          
[121] "1999_06_15_1349"           "1999_06_15_1350"          
[123] "1999_06_15_1352"           "1999_06_15_1353"          
[125] "1999_06_15_1354"           "1999_06_15_1355"          
[127] "1999_06_15_1356"           "1999_06_15_1357"          
[129] "1999_06_15_1358"           

In [80]:
# Rename samples 
################
# Subset metadata 

fastFiles2 <- list.files("data/Dada2_input/")

metadata.f1 <- metadata.f[metadata.f$FilesKeep %in% "Keep",]

metadata.f1$OriginalID <- gsub("-", "_", metadata.f1$OriginalID)

for (i in 1:nrow(metadata.f1)) {
    
    fg <- metadata.f1$OriginalID[i]
    
    select.f <- fastFiles2[grep(fg, fastFiles2)]
    
    r1 <- paste0("data/Dada2_input/", select.f[grep("R1", select.f)])
    r1.new <- paste0("data/Dada2_input/", metadata.f1$NewId[i], "_R1", ".fastq.gz")
    file.rename(from = r1, to = r1.new)
    
    r2 <- paste0("data/Dada2_input/", select.f[grep("R2", select.f)])
    r2.new <- paste0("data/Dada2_input/", metadata.f1$NewId[i], "_R2", ".fastq.gz")
    file.rename(from = r2, to = r2.new)
    }

In [81]:
# Save final version of metadata 
write.csv(metadata.f1[,-c(1,2)], "data/metadata/samples_metadata_f.csv")